# Radar Data Streaming and Pre-Processing with data acquisiton

#### Importing required libraries 

    - ifxdaq --> is the python wrapper for Radar SDK
    - processing --> is python module for basic processing to get the range-Dopplers from the signal
    - fft --> contains the range_fft and doppler_fft methods
    

In [1]:
import ifxdaq
import processing
import numpy as np
#print(ifxdaq.__version__)
from ifxdaq.sensor.radar_ifx import RadarIfxAvian
import matplotlib.pyplot as plot
import os

#### In the folder 'radar_configs' you find radar configuration parameters. It contains radar parameters :

    - transmit power level
    - start/end frequency
    - sampling rate
    - # of chirps per frame
    - # of samples per chirp
    - # of transmit/receive antennas
    - MIMO mode selection

In [2]:
config_file = "radar_configs/RadarIfxBGT60.json"
number_of_frames = 1000


In [3]:
## Run this to understand the current radar settings better
import json
with open(config_file) as json_file:
    c = json.load(json_file)["device_config"]["fmcw_single_shape"]
    chirp_duration = c["num_samples_per_chirp"]/c['sample_rate_Hz']
    frame_duration = (chirp_duration + c['chirp_repetition_time_s']) * c['num_chirps_per_frame']
    print("With the current configuration, the radar will send out " + str(c['num_chirps_per_frame']) + \
          ' signals with varying frequency ("chirps") between ' + str(c['start_frequency_Hz']/1e9) + " GHz and " + \
          str(c['end_frequency_Hz']/1e9) + " GHz.")
    print('Each chirp will consist of ' + str(c["num_samples_per_chirp"]) + ' ADC measurements of the IF signal ("samples").')
    print('A chirp takes ' + str(chirp_duration*1e6) + ' microseconds and the delay between the chirps is ' + str(c['chirp_repetition_time_s']*1e6) +' microseconds.')
    print('With a total frame duration of ' + str(frame_duration*1e3) + ' milliseconds and a delay of ' + str(c['frame_repetition_time_s']*1e3) + ' milliseconds between the frame we get a frame rate of ' + str(1/(frame_duration + c['frame_repetition_time_s'])) + ' radar frames per second.')

With the current configuration, the radar will send out 64 signals with varying frequency ("chirps") between 60.5 GHz and 61.5 GHz.
Each chirp will consist of 128 ADC measurements of the IF signal ("samples").
A chirp takes 64.0 microseconds and the delay between the chirps is 400.0 microseconds.
With a total frame duration of 29.696 milliseconds and a delay of 50.0 milliseconds between the frame we get a frame rate of 12.547681188516362 radar frames per second.


## Function for saving the data

In [4]:
def save_data(raw, rd_map, out_dir=r"out_data"):
    # Creating output directory
    files = os.listdir(out_dir)
    i = 100
    while f"experiment_{i:03d}" in files:
        i+=1
    out_dir = out_dir+f"\experiment_{i:03d}"
    os.makedirs(out_dir)

    # Saving data
    file_name = out_dir + "\\raw_data.npy"
    with open(file_name, 'wb') as outfile:
        np.save(outfile,raw)
    file_name = out_dir + "\\range_dopler_map.npy"
    with open(file_name, 'wb') as outfile:
        np.save(outfile,rd_map)
    print(out_dir)

### Main part of the program

In [ ]:
import time

number_of_frames = 1000
raw_data    = []

# time.sleep(3)

with RadarIfxAvian(config_file) as device:                             # Initialize the radar with configurations
    
    for i_frame, frame in enumerate(device):                           # Loop through the frames coming from the radar
        
        raw_data.append(np.squeeze(frame['radar'].data/(4095.0)))      # Dividing by 4095.0 to scale the data
        if(i_frame == number_of_frames-1):
            data = np.asarray(raw_data)
            range_doppler_map = processing.processing_rangeDopplerData(data)
            #del data
            break   
            
save_data(data, range_doppler_map)